# Teil 4 Demo 1: Abfragen mit SQL

## Bibliotheken & Konfiguration

In [1]:
packages <- c("readr", "DBI", "RSQLite")
install.packages(setdiff(packages, rownames(installed.packages())))
lapply(packages, require, character.only = TRUE)

base_url <- "https://raw.githubusercontent.com/ganslats/TMF-School-2022-Block-4/master/Rohdaten/mimic-iii-demo/"

Loading required package: readr

Loading required package: DBI

Loading required package: RSQLite



[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

## Ausgewählte MIMIC III-Rohdaten laden

In [2]:
mimic.patients.raw      <- read_csv(paste(base_url, "PATIENTS.csv", sep=""),
                                    col_types = cols(row_id = col_double(), subject_id = col_double(), gender = col_character(), dob = col_datetime(format = ""), dod = col_datetime(format = ""), dod_hosp = col_datetime(format = ""), dod_ssn = col_datetime(format = ""), expire_flag = col_double()))
mimic.admissions.raw    <- read_csv(paste(base_url, "ADMISSIONS.csv", sep=""),
                                    col_types = cols(  row_id = col_double(), subject_id = col_double(), hadm_id = col_double(), admittime = col_datetime(format = ""), dischtime = col_datetime(format = ""), deathtime = col_datetime(format = ""), admission_type = col_character(), admission_location = col_character(), discharge_location = col_character(), insurance = col_character(), language = col_character(), religion = col_character(), marital_status = col_character(), ethnicity = col_character(), edregtime = col_datetime(format = ""), edouttime = col_datetime(format = ""), diagnosis = col_character(), hospital_expire_flag = col_double(), has_chartevents_data = col_double()))
mimic.prescriptions.raw <- read_csv(paste(base_url, "PRESCRIPTIONS.csv", sep=""),
                                    col_types = cols(row_id = col_double(), subject_id = col_double(), hadm_id = col_double(), icustay_id = col_double(), startdate = col_datetime(format = ""), enddate = col_datetime(format = ""), drug_type = col_character(), drug = col_character(), drug_name_poe = col_character(), drug_name_generic = col_character(), formulary_drug_cd = col_character(), gsn = col_character(), ndc = col_character(), prod_strength = col_character(), dose_val_rx = col_character(), dose_unit_rx = col_character(), form_val_disp = col_character(), form_unit_disp = col_character(), route = col_character()))
head(mimic.patients.raw)
head(mimic.admissions.raw)
head(mimic.prescriptions.raw)

row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
<dbl>,<dbl>,<chr>,<dttm>,<dttm>,<dttm>,<dttm>,<dbl>
9467,10006,F,2094-03-05,2165-08-12,2165-08-12,2165-08-12,1
9472,10011,F,2090-06-05,2126-08-28,2126-08-28,NA,1
9474,10013,F,2038-09-03,2125-10-07,2125-10-07,2125-10-07,1
9478,10017,F,2075-09-21,2152-09-12,NA,2152-09-12,1
9479,10019,M,2114-06-20,2163-05-15,2163-05-15,2163-05-15,1
9486,10026,F,1895-05-17,2195-11-24,NA,2195-11-24,1


row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
<dbl>,<dbl>,<dbl>,<dttm>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dttm>,<dttm>,<chr>,<dbl>,<dbl>
12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NA,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NA,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1
12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NA,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NA,NA,HEPATITIS B,1,1
12265,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NA,CATHOLIC,NA,UNKNOWN/NOT SPECIFIED,NA,NA,SEPSIS,1,1
12269,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,NA,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NA,CATHOLIC,DIVORCED,WHITE,2149-05-26 12:08:00,2149-05-26 19:45:00,HUMERAL FRACTURE,0,1
12270,10019,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,2163-05-15 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NA,CATHOLIC,DIVORCED,WHITE,NA,NA,ALCOHOLIC HEPATITIS,1,1
12277,10026,103770,2195-05-17 07:39:00,2195-05-24 11:45:00,NA,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NA,OTHER,NA,WHITE,2195-05-17 01:49:00,2195-05-17 08:29:00,STROKE/TIA,0,1


row_id,subject_id,hadm_id,icustay_id,startdate,enddate,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,prod_strength,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,route
<dbl>,<dbl>,<dbl>,<dbl>,<dttm>,<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
32600,42458,159647,NA,2146-07-21,2146-07-22,MAIN,Pneumococcal Vac Polyvalent,Pneumococcal Vac Polyvalent,PNEUMOcoccal Vac Polyvalent,PNEU25I,048548,00006494300,25mcg/0.5mL Vial,0.5,mL,1,VIAL,IM
32601,42458,159647,NA,2146-07-21,2146-07-22,MAIN,Bisacodyl,Bisacodyl,Bisacodyl,BISA5,002947,00536338101,5 mg Tab,10,mg,2,TAB,PO
32602,42458,159647,NA,2146-07-21,2146-07-22,MAIN,Bisacodyl,Bisacodyl,Bisacodyl (Rectal),BISA10R,002944,00574705050,10mg Suppository,10,mg,1,SUPP,PR
32603,42458,159647,NA,2146-07-21,2146-07-22,MAIN,Senna,Senna,Senna,SENN187,019964,00904516561,1 Tablet,1,TAB,1,TAB,PO
32604,42458,159647,NA,2146-07-21,2146-07-21,MAIN,Docusate Sodium (Liquid),Docusate Sodium (Liquid),Docusate Sodium (Liquid),DOCU100L,003017,00121054410,100mg UD Cup,100,mg,1,UDCUP,PO
32605,42458,159647,NA,2146-07-21,2146-07-22,MAIN,Heparin,Heparin,Heparin Sodium,HEPA5I,006549,63323026201,5000 Units / mL- 1mL Vial,5000,UNIT,1,mL,SC


## Rohdaten in SQLite-Datenbank (im Speicher) laden

In [3]:
con <- dbConnect(RSQLite::SQLite(), ":memory:")
dbWriteTable(con, "patients",      mimic.patients.raw)
dbWriteTable(con, "admissions",    mimic.admissions.raw)
dbWriteTable(con, "prescriptions", mimic.prescriptions.raw)
dbListTables(con)

[1] "admissions"    "patients"      "prescriptions"

## Alle Spalten einer Tabelle abfragen

In [4]:
head(dbGetQuery(con, "
SELECT *
  FROM patients
"), 6)

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,9467,10006,F,3918585600,6172934400,6172934400,6172934400,1
2,9472,10011,F,3800304000,4943548800,4943548800,NA,1
3,9474,10013,F,2167084800,4915468800,4915468800,4915468800,1
4,9478,10017,F,3336249600,5765385600,NA,5765385600,1
5,9479,10019,M,4558896000,6102086400,6102086400,6102086400,1
6,9486,10026,F,-2355004800,7128604800,NA,7128604800,1


## Ausgewählte Spalten einer Tabelle abfragen

In [5]:
head(dbGetQuery(con, "
SELECT subject_id,
       gender
  FROM patients
"))

,subject_id,gender
,<dbl>,<chr>
1,10006,F
2,10011,F
3,10013,F
4,10017,F
5,10019,M
6,10026,F


## Ausgewählte Zeilen einer Tabelle abfragen

In [6]:
head(dbGetQuery(con, "
SELECT *
  FROM patients
 WHERE gender = 'M'
"))

,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag
,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,9479,10019,M,4558896000,6102086400,6102086400,6102086400,1
2,9489,10029,M,2880316800,5387472000,NA,5387472000,1
3,9491,10032,M,2532124800,5313686400,5313686400,5313686400,1
4,9494,10035,M,2628115200,5151427200,NA,5151427200,1
5,9501,10042,M,3355948800,5709312000,NA,5709312000,1
6,9502,10043,M,4394736000,6977318400,NA,6977318400,1


## Ergebnis auf eindeutige Datensätze reduzieren (DISTINCT)

In [7]:
head(dbGetQuery(con, "
SELECT DISTINCT
       gender
  FROM patients
"))

,gender
,<chr>
1,F
2,M


## Einfache Aggregation: Gesamtzahl der Datensätze einer Tabelle abfragen

In [8]:
head(dbGetQuery(con, "
SELECT COUNT(*) AS n
  FROM patients
"))

,n
,<int>
1,100


## Aggregation & Gruppierung: Anzahl nach Geschlecht ermitteln

In [9]:
head(dbGetQuery(con, "
SELECT gender,
       COUNT(*) AS n
  FROM patients
 GROUP BY gender
"))

,gender,n
,<chr>,<int>
1,F,55
2,M,45


## Zeilen nach der Aggregation filtern

In [10]:
head(dbGetQuery(con, "
SELECT gender,
       COUNT(*) AS n
  FROM patients
 GROUP BY gender
HAVING COUNT(*) > 50
"))

,gender,n
,<chr>,<int>
1,F,55


## Tabellen per JOIN miteinander verknüpfen

In [11]:
head(dbGetQuery(con, "
SELECT pat.subject_id,
       pat.gender,
       adm.diagnosis
  FROM patients   pat
  JOIN admissions adm ON pat.subject_id = adm.subject_id
"))

,subject_id,gender,diagnosis
,<dbl>,<chr>,<chr>
1,10006,F,SEPSIS
2,10011,F,HEPATITIS B
3,10013,F,SEPSIS
4,10017,F,HUMERAL FRACTURE
5,10019,M,ALCOHOLIC HEPATITIS
6,10026,F,STROKE/TIA


## Patient:innen mit mehr als einem Intensiv-Aufenthalt abfragen

In [12]:
head(dbGetQuery(con, "
SELECT pat.subject_id,
       COUNT(*) AS n_admissions
  FROM patients   pat
  JOIN admissions adm ON pat.subject_id = adm.subject_id
 GROUP BY pat.subject_id
HAVING COUNT(*) > 1
 ORDER BY COUNT(*) DESC
"))

,subject_id,n_admissions
,<dbl>,<int>
1,41976,15
2,44083,3
3,10088,3
4,43881,2
5,42346,2
6,42135,2


## 2 Subsets von Verschreibungen für die Demo verschiedener Joins erzeugen

* Patient:innenen mit Hauptdiagnose Sepsis
* Patient:innen mit Gabe von Vancomycin (Reserve-Antibiotikum z.B. bei MRSA)

In [13]:
dbExecute(con, "CREATE TABLE demo_sepsis     AS SELECT DISTINCT subject_id FROM admissions    WHERE LOWER(diagnosis) LIKE '%sepsis%'")
dbExecute(con, "CREATE TABLE demo_vancomycin AS SELECT DISTINCT subject_id FROM prescriptions WHERE LOWER(drug)      LIKE '%vancomycin%'")

[1] 0

[1] 0

## Patient:innen abfragen, die sowohl eine Sepsis als Hauptdiagnose hatten als auch Vancomycin erhalten haben (INNER JOIN)

In [14]:
head(dbGetQuery(con, "
SELECT *
  FROM demo_sepsis     sep
  JOIN demo_vancomycin van on sep.subject_id = van.subject_id
"), 100)

,subject_id,subject_id
,<dbl>,<dbl>
1,10006,10006
2,10036,10036
3,10056,10056
4,10088,10088
5,10094,10094
6,41976,41976


## Alle Patient:innen abfragen, die eine Sepsis als Hauptdiagnose hatten sowie die mit Vancomycingabe ergänzen (LEFT OUTER JOIN)

In [15]:
head(dbGetQuery(con, "
SELECT *
  FROM      demo_sepsis     sep
  LEFT JOIN demo_vancomycin van on sep.subject_id = van.subject_id
"), 100)

,subject_id,subject_id
,<dbl>,<dbl>
1,10006,10006
2,10013,NA
3,10036,10036
4,10056,10056
5,10088,10088
6,10094,10094
7,40601,NA
8,41976,41976


## Alle Patient:innen abfragen, die eine Vancomycingabe hatten und die keine Sepsisdiagnose hatten (RIGHT OUTER JOIN)

### Hinweis: SQLite unterstützt keine RIGHT JOINS, deshalb ist das hier als LEFT JOIN mit vertauschter Reihenfolge dargestellt

In [16]:
head(dbGetQuery(con, "
SELECT *
  FROM      demo_vancomycin van
  LEFT JOIN demo_sepsis     sep on van.subject_id = sep.subject_id
"), 100)

,subject_id,subject_id
,<dbl>,<dbl>
1,43735,NA
2,10006,10006
3,10088,10088
4,10056,10056
5,10065,NA
6,44228,NA
7,10124,NA
8,10111,NA
9,10036,10036


## Tatsächliche Hauptdiagnosen für die Patient:innen mit Vancomycingabe ohne Sepsis ermitteln

In [17]:
head(dbGetQuery(con, "
SELECT adm.diagnosis,
       COUNT(*)
  FROM      demo_vancomycin van
  JOIN      admissions      adm ON van.subject_id = adm.subject_id
  LEFT JOIN demo_sepsis     sep on van.subject_id = sep.subject_id
 WHERE sep.subject_id IS NULL
 GROUP BY adm.diagnosis
 ORDER BY COUNT(*) DESC
"), 20)

,diagnosis,COUNT(*)
,<chr>,<int>
1,FEVER,3
2,UPPER GI BLEED,2
3,PNEUMONIA,2
4,LIVER FAILURE,2
5,HYPOTENSION,2
6,GASTROINTESTINAL BLEED,2
7,FAILURE TO THRIVE,2
8,CONGESTIVE HEART FAILURE,2
9,ASTHMA;CHRONIC OBST PULM DISEASE,2


## Datenbankverbindung schließen

In [18]:
dbDisconnect(con)